# Instacart - PROJECT NAME

## Data cleaning process

Libraries.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Step 1

Load datasets and preprocess data. Instacart customer purchase information is divided into 6 datasets:

1. Aisles
2. Departments
3. Order_products__prior (This table includes training orders and indicates whether a product in an order is a reorder or not (through the reordered variable)
5. Orders
6. Products

I have perfomed some join operations with MYSQL to merge Aisles, Departments and Products files. The resulting dataset is named info.csv.

In [2]:
info = pd.read_csv('../data/info.csv')
print('-- NULL VALUES --')
print(info.isna().sum(), '\n')
print('-- INFO --')
info.info()
print()
print('-- SUMMARY --')
display(round(info.describe().T,2))
display(round(info.describe(include=object).T,2))
print()
print('-- FIRST 5 ROWS --')
info.head()

-- NULL VALUES --
product_id       21
product_name      0
department_id     0
aisle_id         21
aisle            21
department        0
dtype: int64 

-- INFO --
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32994 entries, 0 to 32993
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_id     32973 non-null  float64
 1   product_name   32994 non-null  object 
 2   department_id  32994 non-null  int64  
 3   aisle_id       32973 non-null  float64
 4   aisle          32973 non-null  object 
 5   department     32994 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 1.5+ MB

-- SUMMARY --


,count,mean,std,min,25%,50%,75%,max
product_id,32973.0,16487.79,9518.91,1.0,8245.0,16488.0,24731.0,32974.0
department_id,32994.0,11.73,5.84,1.0,7.0,13.0,17.0,21.0
aisle_id,32973.0,67.65,38.35,1.0,34.0,69.0,100.0,134.0


,count,unique,top,freq
product_name,32994,32994,Lasagna Bake with Meat Sauce,1
aisle,32973,134,missing,812
department,32994,21,personal care,4382



-- FIRST 5 ROWS --


,product_id,product_name,department_id,aisle_id,aisle,department
0,3647.0,Soft Dinner 6 Ct White Rolls,1,58.0,frozen breads doughs,frozen
1,3646.0,Eggo Breakfast Sandwich Bacon Egg Cheese,1,52.0,frozen breakfast,frozen
2,3645.0,Creamed Spinach,1,116.0,frozen produce,frozen
3,3628.0,Gluten Free Paleo Fettuccine,1,38.0,frozen meals,frozen
4,3608.0,Brie & Raspberry Bake & Serve Fillo Rolls,1,129.0,frozen appetizers sides,frozen


In [3]:
info['product_id'].nunique()

32973

*Cols meaning*
* order_dow: Day Of Week
* eval_set: which evaluation set this order belongs in (see train and prior dataset described below)
* order_number: the order sequence number for this user (1 = first, n = nth)
* days_since_prior: days since the last order, capped at 30 (with NAs for order_number = 1)
* add_to_cart_order: order in which each product was added to cart
* reordered: 1 if this product has been ordered by this user in the past, 0 otherwise
* “prior”: orders prior to that users most recent order

#### Merging orders files.

In [4]:
orders = pd.read_csv('../data/orders.csv')
print('-- NULL VALUES --')
print(orders.isna().sum(), '\n')
print('-- INFO --')
orders.info()
print()
print('-- SUMMARY --')
display(round(orders.describe().T,2))
display(round(orders.describe(include=object).T,2))
print()
print('-- FIRST 5 ROWS --')
orders.head()

-- NULL VALUES --
order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64 

-- INFO --
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               int64  
 5   order_hour_of_day       int64  
 6   days_since_prior_order  float64
dtypes: float64(1), int64(5), object(1)
memory usage: 182.7+ MB

-- SUMMARY --


,count,mean,std,min,25%,50%,75%,max
order_id,3421083.0,1710542.00,987581.74,1.0,855271.5,1710542.0,2565812.5,3421083.0
user_id,3421083.0,102978.21,59533.72,1.0,51394.0,102689.0,154385.0,206209.0
order_number,3421083.0,17.15,17.73,1.0,5.0,11.0,23.0,100.0
order_dow,3421083.0,2.78,2.05,0.0,1.0,3.0,5.0,6.0
order_hour_of_day,3421083.0,13.45,4.23,0.0,10.0,13.0,16.0,23.0
days_since_prior_order,3214874.0,11.11,9.21,0.0,4.0,7.0,15.0,30.0


,count,unique,top,freq
eval_set,3421083,3,prior,3214874



-- FIRST 5 ROWS --


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [5]:
order_products_prior = pd.read_csv('../data/order_products__prior.csv')
print('-- NULL VALUES --')
print(order_products_prior.isna().sum(), '\n')
print('-- INFO --')
order_products_prior.info()
print()
print('-- SUMMARY --')
display(round(order_products_prior.describe().T,2))
print()
print('-- FIRST 5 ROWS --')
display(order_products_prior.head())

MemoryError: Unable to allocate 990. MiB for an array with shape (4, 32434489) and data type int64

In [ ]:
orders_full = pd.merge(orders, order_products_prior, on = 'order_id')
print('-- NULL VALUES --')
print(orders_full.isna().sum(), '\n')
print('-- INFO --')
orders_full.info()
print()
print('-- SUMMARY --')
# display(round(orders_full.describe().T,2))
print()
print('-- FIRST 5 ROWS --')
display(orders_full.head())

### Merging info file and orders file.

In [ ]:
info_orders = pd.merge(orders_full, info, on = 'product_id')
print('-- NULL VALUES --')
print(info_orders.isna().sum(), '\n')
print('-- INFO --')
info_orders.info()
print()
print('-- SUMMARY --')
display(round(info_orders.describe().T,2))
print()
print('-- FIRST 5 ROWS --')
display(info_orders.head())

In [ ]:
sorted_instacart = info_orders.sort_values(by='order_id').reset_index(drop=True)
sorted_instacart.tail()

In [ ]:
sorted_instacart = sorted_instacart.drop('eval_set', axis = 1)
instacart_sample = sorted_instacart[sorted_instacart['user_id'] <= 200000].reset_index(drop=True)

In [ ]:
instacart_sample.to_csv('../data/instacart_sample.csv')

## EDA

* Time period
* How many products?
* Boxplot distribution of products

#### How many users and orders?

In [ ]:
print('Number of users:', instacart_sample['user_id'].nunique())
print('Total number of orders:', instacart_sample['order_id'].nunique())

#### How many purchases (orders) by user?

In [ ]:
order_by_user = instacart_sample.groupby('user_id')['order_number'].max()
order_by_user_df = pd.Series.to_frame(order_by_user) 
order_by_user_df.head()

In [ ]:
print('Max number of orders by user:', max(order_by_user_df['order_number']))
print('Min number of orders by user:', min(order_by_user_df['order_number']))
print('Median number of orders by user:', round(np.median(order_by_user_df['order_number'])))
print('Mean number of orders by user:', round(np.mean(order_by_user_df['order_number'])))

In [ ]:
fig, ax = plt.subplots(figsize = (40,25), dpi = 100) 
sns.countplot(x=order_by_user_df['order_number'])
ax.set(xlabel='Number of orders', ylabel='Users')
plt.show()

#### Most 'popular' day

In [ ]:
orders_day = instacart_sample.groupby('order_id')['order_dow'].max()
orders_day

In [ ]:
fig, ax = plt.subplots(figsize = (40,25), dpi = 100) 
sns.countplot(x=orders_day.values)
ax.set(xlabel='Days of the Week', ylabel='Number of orders')
plt.show()

#### Most 'popular' hour

In [ ]:
orders_hour = instacart_sample.groupby('order_id')['order_hour_of_day'].max()
orders_hour

In [ ]:
fig, ax = plt.subplots(figsize = (40,25), dpi = 100) 
sns.countplot(x=orders_hour.values)
ax.set(xlabel='Hours of the day', ylabel='Number of orders')
plt.show()

#### How many days goes by between orders?

In [ ]:
reorders_time = instacart_sample.groupby('order_id')['days_since_prior_order'].max()
reorders_time

In [ ]:
fig, ax = plt.subplots(figsize = (40,25), dpi = 100) 
sns.countplot(x=reorders_time.values)
ax.set(xlabel='Days between orders', ylabel='Number of orders')
plt.show()

#### Most ordered products

In [ ]:
top_20_ordered_prod = instacart_sample[['product_name', 'aisle']].value_counts().head(20)
pd.Series.to_frame(top_20_ordered_prod)

#### Most reordered products

In [ ]:
reordered_prod = instacart_sample.loc[instacart_sample['reordered'] == 1]
top20_products = reordered_prod[['product_name','aisle']].value_counts().head(200)
pd.Series.to_frame(top20_products)

#### Most visited 'aisles'

In [ ]:
top10_aisles = instacart_sample['aisle'].value_counts().head(10)
top10_aisles

In [ ]:
labels = ['fresh fruits', 'fresh vegetables', 'packaged vegetables fruits', 'yogurt', 'milk', 'packaged cheese',
         'water seltzen sparkling water', 'chips pretzels', 'bread', 'soy lactosefree']
ax, fig = plt.subplots(figsize=(15,10))
plt.pie(top10_aisles, labels = labels, autopct='%.0f%%')
plt.show()

### Time period: one year

Could we infere the period time? If a customer has ordered 12 times with a frequency of XX days, we could know the period of time for our data. Let's compute the sum of days for each customer to check the maximum value of days_since_prior_order and get the time range.

In [ ]:
orders.head()

In [ ]:
orders.loc[orders['days_since_prior_order'] == 0]

In [ ]:
sum_days_orders = orders.groupby('user_id')['days_since_prior_order'].sum()
max(sum_days_orders)